In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

# Predicting droughts and weather from weather data
The International Crops Research Institute for the Semi-Arid Tropics (ICRISAT) is an international organization which conducts agricultural research for rural development, headquartered in Hyderabad, Telangana, India.

Founded in 1972 by a consortium of organisations convened by the Ford and the Rockefeller foundations, ICRISAT’s charter was signed by The Food and Agriculture Organization of the United Nations (FAO) and The United Nations Development Programme (UNDP).

ICRISAT has been collecting detailed daily weather data in Hyderabad since 1978, and published a 40-year dataset of daily weather readings. I will use this dataset as the first dataset for my analysis.

## Datasets
1. ICRISAT daily weather dataset for 1978-2018, collected in Hyderabad.
1. Integrated drought index data monthly dataset for 1951-2016, of Hyderabad.

## Questions
1. Which, if any, of the weather features collected by ICRISAT appear to be correlated with each other?
1. Which months had the highest mean and median rainfall, humidity and radiation? Which months had the lowest?
1. Can the weather features collected indicate or predict drought using a model?


Firstly, I will write a function to load data from the Vocareum folder into a pandas DataFrame.

The `load_data()` function also prints `DataFrame.info()`, `DataFrame.describe()` and `DataFrame.head()` to provide a sense of what the dataset contains.

In [ ]:
def load_data(file: Path, **kwargs):
    '''
    Wrapper function to
        1. Loads data into a DataFrame, and
        2. Prints df.info() and df.describe().
    :param file: Path of file location
    :param kwargs: kwargs to pass to pd.read_csv() or pd.read_excel()
    :return: pd.DataFrame
    '''
    # reads data into df then and prints description and info
    if 'xls' in file.suffix:
        df = pd.read_excel(file, **kwargs)
    elif 'csv' in file.suffix:
        df = pd.read_csv(file, **kwargs)
    else:
        print(file.suffix)
        raise NotImplementedError

    print(f"===== df.info output =====")
    df.info()
    print(f"\n\n===== df.describe output =====\n", df.describe())
    print("\n\n===== df.head output =====\n", df.head())
    return df

## Dataset: ICRISAT weather observations

In [ ]:
cwd = Path.cwd()  # reads current working directory to simplify working with files
weather_df = load_data(cwd / "data" / "ICRISAT Weather 1978 to 2018.xlsx")

### Dataset: ICRISAT - fields
The ICRISAT data is at the day level and includes the following fields:

| #   | Column Name        | Description                                   |
|-----|:-------------------|:----------------------------------------------|
| 0   | Station            | Single value "ICRISAT"                        |
| 1   | Date               | Date of collection of weather data            |
| 2   | MaxT               | Maximum temperature (°C)                      |
| 3   | MinT               | Minimum temperature (°C)                      |
| 4   | RH1                | Relative humidity in the morning (%)          |
| 5   | RH2                | Relative humidity in the afternoon (%)        |
| 6   | Wind               | Wind (km/h)                                   |
| 7   | Rain               | Rain (mm)                                     |
| 8   | SSH                | Bright sunshine (hour)                        |
| 9   | Evap               | Evaporation (mm)                              |
| 10  | Radiation          | Radiation (mm/hour)                           |
| 11  | FAO56<sub>ET</sub> | FAO 56 reference crop evapotranspiration (mm) |
| 12  | Lat                | Latitude of collection point                  |
| 13  | Lon                | Longitude of collection point                 |
| 14  | Cum<sub>Rain</sub> | Cumulative monthly rainfall (mm)              |


### Dataset: ICRISAT - Preview of data

In [ ]:
weather_df.sample(10, random_state=42).sort_index()

### Dataset: ICRISAT - location of station
The coordinates of the ICRISAT research station are at:

In [ ]:
coordinates = (weather_df.loc[0, 'Lat'], weather_df.loc[0, 'Lon'])
coordinates

Let's visualise this location on a map:

In [ ]:
try:
    fig = go.Figure(px.scatter_mapbox(weather_df, lat='Lat', lon='Lon', mapbox_style='open-street-map'))
    fig.show()
except SyntaxError:
    from IPython.display import Image
    Image(cwd / "data" / "fallback-image-icrisat.jpg")

## Dataset: Integrated Drought Index (IDI)

Drought is a multifaceted weather phenomenon and can be characterised as meteorological, agricultural, hydrological, and groundwater droughts.

A drought index can be used to characterize drought, typically using gridded maps at regional and national levels. These indices help to characterise the different types of drought and quantify severity levels as well as onset and termination of drought.

The Integrated Drought Index (IDI) dataset is a drought index developed by Shah & Mishra (2020) that integrates the effects of these types of droughts into a single indicator.

Shah & Mishra (2020) has published monthly IDI from their study in 0.25° grids. The closest grid to the ICRISAT station was selected and read below into `drought_df`.

In [ ]:
drought_df = load_data(cwd / "data" / "data_17.625_78.375.csv", header=None,
                       delim_whitespace=True, names=['year', 'month', 'idi'])

## Dataset: IDI - fields

The IDI data is at the month level and includes the following fields:

| #   | Column Name | Description                            |
|-----|:------------|:---------------------------------------|
| 0   | Year        | Year                                   |
| 1   | Month       | Month                                  |
| 2   | IDI         | Integrated Drought Index - index value |

## Dataset: IDI - Preview of data

In [ ]:
drought_df.sample(10, random_state=42).sort_index()

# Question 1
>Which, if any, of the weather features collected by ICRISAT are related to each other?

To answer this, I will show a correlation matrix of the ICRISAT dataset.

This can be done directly in a pandas dataframe with `DataFrame.corr()`. To increase visual understanding, I will plot it with a `seaborn` heatmap.

In [ ]:
sn.heatmap(weather_df.corr(), annot=True)
plt.show()

I will also query the correlation matrix directly to obtain descriptive statistics, and the top five correlated pairs of variables.

In [ ]:
corr_matrix = (weather_df
               .corr()
               .stack()
               .reset_index()
               .rename(columns={'level_0': 'var1', 'level_1': 'var2', 0: 'value'})
               .query("var1 != var2 and var1 < var2")
               )

corr_matrix.describe()

In [ ]:
(corr_matrix
 .sort_values(by='value', ascending=False)
 .head(5)
 .reset_index(drop=True)
 )

## Question 1 - results

Most variables of the ICRISAT dataset are not highly correlated with each other.

The top 5 pairs of variables that exhibit correlation are:
1. Evaporation (mm) and FAO 56 reference crop evapotranspiration (mm)
2. Evaporation (mm) and Maximum temperature (°C)
3. FAO 56 reference crop evapotranspiration (mm) and Maximum temperature (°C)
4. FAO 56 reference crop evapotranspiration (mm) and Radiation (mm/hour)
5. Radiation (mm/hour) and Bright sunshine (hour)

# Question 2
> Which months had the highest average and median rainfall, humidity and radiation? Which months had the lowest?

First, I create two other dataframes aggregated with mean and median.

In [ ]:
weather_df_monthly_mean = (weather_df
                           .resample('M', on='Date').mean()
                           .reset_index()
                           .rename(columns={'Date': 'Month'})
                           )

weather_df_monthly_median = (weather_df
                             .resample('M', on='Date').median()
                             .reset_index()
                             .rename(columns={'Date': 'Month'})
                             )

Firstly, create some convenience functions to print the highest and lowest values and months.

If multiple months exist with the same highest or lowest value, the first month will be printed.

In [ ]:
def print_weather_stats(agg_type, weather_var):
    '''
    Prints weather stats from `weather_df`
    :param agg_type: accepts either 'mean' or 'median'
    :param weather_var: accepts a column in `weather_df` with weather statistics.
    :return: None
    '''
    dict_agg_type = {'mean': weather_df_monthly_mean, 'median': weather_df_monthly_median}
    _a, _b = dict_agg_type.get(agg_type).loc[dict_agg_type.get(agg_type)[weather_var].idxmax(), ['Month', weather_var]]
    print(f"{_a:%B %Y} had the {agg_type} highest {weather_var} of {_b:.2f}.")
    _c, _d = dict_agg_type.get(agg_type).loc[dict_agg_type.get(agg_type)[weather_var].idxmin(), ['Month', weather_var]]
    print(f"{_c:%B %Y} had the {agg_type} lowest {weather_var} of {_d:.2f}.")
    return

## Question 2 - results
### Rainfall

In [ ]:
print_weather_stats('median', 'Rain')
print_weather_stats('mean', 'Rain')

### Humidity

In [ ]:
print_weather_stats('median', 'RH1')
print_weather_stats('mean', 'RH1')

### Radiation

In [ ]:
print_weather_stats('median', 'Radiation')
print_weather_stats('mean', 'Radiation')

# Question 3
> Can the weather features collected indicate or predict drought using a model?

It will be interesting to see whether ICRISAT weather data can be used to predict droughts.

First, I will do some exploratory data analysis on the features exhibiting correlation.

As seen below, there is high variability and seasonality in the weather data collected.

In [ ]:
px.line(weather_df, x='Date', y='Evap', title='Evaporation (mm) over time')

In [ ]:
px.line(weather_df, x='Date', y='FAO56_ET', title='FAO 56 reference crop evapotranspiration (mm) over time')

In [ ]:
px.line(weather_df, x='Date', y='MaxT', title='Max temperature (°C) over time')

In [ ]:
px.line(weather_df, x='Date', y='Radiation', title='Radiation (mm/hour) over time')

In [ ]:
px.line(weather_df, x='Date', y='SSH', title='Bright sunshine (hour) over time')

As `weather_df` and `drought_df` are at different granularity, I will now resample `drought_df` to the daily granularity.

This maintains the daily data in `weather_df`.

In [ ]:
# first, make timestamps from the year and month columns
drought_df_daily = (drought_df
                    .astype({'year': str, 'month': str})
                    .assign(ym=lambda x: x.year + x.month.str.pad(2, side='left', fillchar='0') + '01',
                            ds=lambda x: pd.to_datetime(x.ym, format='%Y%m%d')
                            )
                    .set_index('ds')
                    )

In [ ]:
# next, upsample monthly data to daily, using method described
# in https://stackoverflow.com/questions/29612705/converting-pandas-dataframe-from-monthly-to-daily
start_date = drought_df_daily.index.min() - pd.DateOffset(day=1)
end_date = drought_df_daily.index.max() + pd.DateOffset(day=31)
dates = pd.date_range(start_date, end_date, freq='D')
dates.name = 'Date'
drought_df_daily = (drought_df_daily
                    .reindex(dates, method='ffill')
                    .reset_index()
                    .drop(['year', 'month', 'ym'], axis=1)
                    )
drought_df_daily.sample(12, random_state=42)

The resampled IDI dataframe (`drought_df_daily`) is then merged with the `weather_df` dataframe.

Following Shah & Mishra (2020), IDI index values of less than -0.8 are indicated as `True`, _i.e._,  drought, in the merged dataframe.

In [ ]:
# then, merge weather_df with drought_df_daily and create a new boolean feature `is_drought`
# `is_drought` indicates a drought condition if `idi` < -0.8
weather_idi_df = (weather_df
                  .merge(drought_df_daily, on='Date')
                  .assign(is_drought = lambda x: x.idi < -0.8)
                  .drop(['Station', 'Lat', 'Lon', 'idi'], axis=1)
                  )

Sample of merged `weather_idi_df` dataframe:

In [ ]:
weather_idi_df.sample(10, random_state=42)

## Setting up test and train datasets

To begin training the models, I first split `weather_idi_df` into: 

* a `X` dataframe (dropping `Date` and target variable) and
* a `y` series (which only contains the target).

Then, `X` and `y` are each split into test and train datasets.

In [ ]:
X, Y = weather_idi_df.drop(['is_drought', 'Date'], axis=1), weather_idi_df['is_drought']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [ ]:
# Inspecting shapes
for ds in [X_train, X_test, y_train, y_test]:
    print(ds.shape)

## Random Forest model

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, oob_score=True)
rf_model.fit(X_train.values, y_train)

### Random Forest model metrics

In [ ]:
print(f"Random Forest OOB score = {rf_model.oob_score_:.5f}")

In [ ]:
print(f"Random Forest model score = {rf_model.score(X_test.values, y_test):.5f}")

In [ ]:
y_probs = rf_model.predict_proba(X_test.values)[:,1]
print(f"Random Forest model ROC AUC score = {metrics.roc_auc_score(y_test, y_probs):.5f}")

## Logistic Regression model

In [ ]:
scaler = StandardScaler()
X_train_scaled, X_test_scaled = scaler.fit_transform(X_train), scaler.fit_transform(X_test)
est = LogisticRegression(solver='liblinear', max_iter=100)

est.fit(X_train, y_train)

### Logistic Regression model metrics

In [ ]:
print(f"Logistic regression score on test dataset = {est.score(X_test, y_test):.5f}")

In [ ]:
print(f"Logistic regression score on train dataset = {est.score(X_train, y_train):.5f}")

### Question 3 - conclusion

Conclusion: the weather features collected by ICRISAT can predict drought with the Random Forest and Logistic Regression models.

For the Random Forest model, the metrics of OOB score, model score and ROC AUC score all exceed 86%, indicating that Random Forest **can** serve to predict drought conditions accurately using observed weather data in the ICRISAT weather dataset.

For the Logistic Regression model, the scores on both train and test dataset are close at 0.785-0.787. While lower than the scores for the Random Forest model, both scores are close together which also indicates that the logistic regression model **can** also produce good predictions using data from the ICRISAT weather dataset.

## Further analysis
The following can be further analysed in a future study:

1. While the above used a simple indicator of drought, can other indicators of drought also be studied, e.g. a more complex indicator can be built, e.g. of _x_ months of IDI remaining below 0?
1. Instead of using daily weather data, can an aggregated IDI also be used at the month level?
1. On top of meteorological drought, can other forms of drought e.g. hydrological and groundwater droughts be studied using other datasets?
1. Can other statistical or machine learning models be used to model and predict droughts?